In [1]:
%matplotlib widget 
# (If running in vscode) and ensure you have "ipympl" module installed in your python environment
from ipywidgets import *
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp 

In [2]:
# ------------- CONSTANTS AND PARAMETERS -------------
# mostly set to 1 to work in a "unitless" regime (so numbers are order 1 in out plots) 
q = 1.0 # charge
E = 1.0 # Electric field E in y direction
B = 1.0 # Magnetic field B in z direction
h = 1.0 # 2h is the plate separation for velocity selector
L = 2.0 # L is length of velocity selector
m = 1.0 # mass, set to 1 usually
mag_initial_velocity = 1 # magnitude of initial velocity vector

# for plotting, sets limits so we can see the detectors
plotting_epsilon_x = 0.2
plotting_epsilon_y = 0.4

In [3]:
def equation_of_motion(t,u,E,B):
    # the RHS of \dot{u} = ... where u = [x,y,v_x,v_y] using the forces present inside the velocity selector
    # t is time (not explicitly used here, needed for solve_ivp), 
    # It is assumed that B = Bz, E = Ey
    return np.array([u[2], u[3], (q/m)*B*u[3], (q/m)*(E - B*u[2])])

def hit_walls(t,u,E,B):
    return np.abs(u[1]) - h # = 0 when wall is hit

# For using the scipy solve_ivp event handler
hit_walls.terminal = True # stop as soon as we cross |y| = h
hit_walls.direction = 1

def hit_detector(t,u,E,B):
    return u[0] - L # = 0 when detector is hit

# For using the scipy solve_ivp event handler
hit_detector.terminal = True # stop as soon as we cross x = L
hit_detector.direction = 1

In [4]:
tmin = 0
tmax = 10   # the size you should set this as depends on what you chose your otehr parameters to be. 
            # Just needs to be "big enough", the event handler will make sure you dont waste time integrating too long
t = np.linspace(tmin,tmax,250) # increase number of points for "smoother" plots

# initial solve
sol = solve_ivp(equation_of_motion, t_span=(tmin,tmax), y0=[0.0,0.0,1.0,0.0], t_eval=t, args=(E,B), events=(hit_walls, hit_detector))

x,y = sol.y[0], sol.y[1]
#plt.plot(x,y)

In [5]:
# Make plot
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
ax.plot(x, y) # Make sure you assign the plot object to a variable

xlims = (0,(1+plotting_epsilon_x)*L)
ylims = (-(1+plotting_epsilon_y)*h,(1+plotting_epsilon_y)*h)

ax.set_xlim(xlims)
ax.set_ylim(ylims)

# Define a function which updates the plot data when slider changes
def update(E = 1.0, B = 1.0, theta=0):
    # theta is deviation from x axis for the direction of initial velocity

    # Completely redraw the axis data
    ax.clear()

    vx0 = mag_initial_velocity*np.cos(theta)
    vy0 = mag_initial_velocity*np.sin(theta)

    sol = solve_ivp(equation_of_motion, t_span=(tmin,tmax), y0=[0.0,0.0,vx0,vy0], t_eval=t, args=(E,B), events=(hit_walls, hit_detector))
    x,y = sol.y[0], sol.y[1]

    #hit_index = np.argmax(np.logical_or((x >= L) , (np.abs(y) >= h)))
    #mask = (x < L) & (np.abs(y) < h) # Only plot when withing the desired bounds

    ax.plot(x,y)
    ax.set_xlim(xlims)
    ax.set_ylim(ylims)

    # Reset plot parameters too
    plt.xlabel("x")
    plt.ylabel("y")
    #plt.title(f"N = {np.sum(total_cut)}")

    fig.canvas.draw_idle()
    fig.tight_layout() # Not entirely necessary

# You may need to run plt.show() here depending on your jupyter setup
#plt.show

# Run the interaction
interact(update,
        E = (-1,3,0.01),
        B = (-1,3,0.01),
        theta = (-np.pi/2,np.pi/2,0.01)
        );

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=1.0, description='E', max=3.0, min=-1.0, step=0.01), FloatSlider(value…